https://adventofcode.com/2022/day/7

In [51]:
test = '''$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
'''

In [64]:
import re

dd={}
lineNo = 0
dirName = '\\'
dd[dirName] = {}
lines = test.split('\n')
while lineNo < len(lines):
    if '$ ls' in lines[lineNo]:
        lineNo += 1
        fileTot = 0
        if re.match(r"\dir \w+",lines[lineNo]):
            dirName = re.findall(r"dir \w+", lines[lineNo])[0].split(' ')[1]
            dd[dirName].
            print(dirName)
        if re.findall(r"\d+",lines[lineNo]):
            fileTot += int(re.findall(r"\d+",lines[lineNo])[0])
            print(fileTot)
        if 
    lineNo += 1


None
None
None
None


In [82]:
print( re.findall(r"\d+", '1234') )

['1234']


The really challenging part about this is how to create the data structure to store the information about each directory. I'm stumped, so let's look to Dr. Google. . .

There's Reddit for AOC: https://www.reddit.com/r/adventofcode/comments/zesk40/2022_day_7_solutions/ that has some suggestions on how to do this in Python.

I've not heard about the Dataclass. This page offers a nice expln: __[https://aoc.just2good.co.uk/python/classes#dataclass]__ before but this looks like it would do the trick. This is a new feature to Python 3.7 and above __[https://realpython.com/python-data-classes/]__

Another coder suggested creating 

In [88]:
from dataclasses import dataclass
@dataclass(frozen=True)
class File:
    """has name and size"""
    name : str
    size : int


In [113]:
from __future__ import annotations

class Directory:
    def __init__(self,name:str) -> None:
        self._name = name
        self._files: list[File] = []
        self._dirs: list[Directory] = []

    @property
    def name(self):
        return self._name

    def add_file(self,a_file: File):
        self._files.append(a_file)
    
    def add_directory(self, a_dir: Directory):
        self._dirs.append(a_dir)
        a_dir.parent_dir = self

    def add_directory(self, a_dir: Directory):
        """ Add a Directory to this directory. Set THIS directory to be its parent dir. """
        self._dirs.append(a_dir)
        a_dir.parent_dir = self 
    
    @property
    def parent_dir(self):
        return self._parent_dir
    
    @parent_dir.setter
    def parent_dir(self, a_dir: Directory):
        self._parent_dir = a_dir

    @property
    def directories(self):
        return self._dirs
    
    def get_directory(self, name: str) -> Directory:
        return next(dir for dir in self.directories if dir.name == name)

    def get_all_dirs(self) -> list[Directory]:
        all_dirs = []
        for a_dir in self.directories:
            all_dirs.extend(a_dir.get_all_dirs())

        all_dirs.extend(self.directories)
        return all_dirs

    @property
    def size(self):
        return sum(file.size for file in self._files) + sum(dir.size for dir in self._dirs)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(name={self.name}, size={self.size}, dirs={len(self.directories)})"



In [111]:
def fs_parse(instructions: list[str]) -> Directory:
    """ Processes instructions, builds directory tree, and returns the root directory.
    
    Lines starting with $ are commands
      $ cd ..
      $ cd some_dir
      $ ls 
    
    Else, lines are listings, which show either:
      sz some_file
      dir some_dir
    """
    root_dir = Directory("/")
    current_dir = root_dir
    
    for line in instructions:
        if line.startswith("$"): # this line is a command
            cmd_line = line.split()
            cmd = cmd_line[1]
            if cmd == "ls":
                continue # we're now in directory list mode, so skip to next line
            if cmd == "cd": # Changing directory
                arg = cmd_line[2]
                if arg == "..": # Going back up a level
                    assert current_dir.parent_dir, "We're not at root"
                    current_dir = current_dir.parent_dir
                else: # Change to named directory
                    if arg != '/': # Changing to named dir. We need find the directory in our list.
                        # It is possible to have multiple directories with the same name.
                        # But directory names are unique within the current directory.
                        current_dir = current_dir.get_directory(arg)
                    else: # Change to root. Only happens once at the top of the instructions.
                        current_dir = root_dir 
            else:
                assert False, "There is no other valid command!"
        else: # we must be dir listing
            ls_line = line.split()
            if ls_line[0] == "dir": # add a new directory
                new_dir = Directory(ls_line[1])
                current_dir.add_directory(new_dir) # add as subdirectory to current dir
            else: # must be a file
                file = File(ls_line[1], size=int(ls_line[0]))
                current_dir.add_file(file)
    
    return root_dir


In [116]:
root_dir = fs_parse(instructions.split('\n'))

In [117]:
all_dirs = root_dir.get_all_dirs()
print(f"All dirs count={len(all_dirs)}.")

All dirs count=158.


In [118]:
small_dirs = [a_dir for a_dir in all_dirs if a_dir.size < 100000]
print(f'\nPart1: Small dirs total = {sum(dir.size for dir in small_dirs)}')


Part1: Small dirs total = 1427048


In [123]:
unused_space = 70000000-root_dir.size
extra_needed = 30000000 - unused_space
extra_needed

2536714

In [124]:
dirs_big_enough = [a_dir for a_dir in all_dirs if a_dir.size >= extra_needed]
smallest_big_dir = min(dirs_big_enough, key=lambda x: x.size)
print(f"part2: smallest dir ={smallest_big_dir.name}: {smallest_big_dir.size}")

part2: smallest dir =snqcgbs: 2940614


Here's another interesting way to store the data. Define the FS as a set of strings, representing all the directorys and files (with the file size at the end of the line). 

__[https://topaz.github.io/paste/#XQAAAQCwBAAAAAAAAAA0m0pnuFI8c/T1e7AfHAKo1XEJwC0R287IfvVPUD9k5IEU6mAvjvZq9BxzZpQ5vhDwQeqG6kvVHlRJ1Q9ZPUiMw5eDVr92LBgWiR4uef00hUBDq+8DXvZeCsmjczU4IYb6PNWMGKIgR+Eim02E2jFF7VdPc3LK/VFZms1wKeeCpZNlkrfVF+g5qM60qQrTF49H7aMp+oFPdHPxoqI5FMMl5oGaXCZHj01Z5b7vhH8J+X44qdvPoVKE0urQxiEGao1jaJkGPvDDz1ahadrp3/b0a5cIL/4mqP/4cm6ele3EuVeIVyzzoZH2VY+VuzLSPo2wGEhejLf41cmdPOJuS1lFj9ovSEgKs0oRLhRTxhZbDIWijSCyS6mdV9DFwv2oW9gUMU8B1kNPzxAvoAQ2q0xegpGZQ+jSZDcCfKlpSmBaTg9OZl/tdAZhhAY7szvUCtnsQknd8OswWHpYnxIyGJBSZ5au1CF1XfyokS6r5LuJFRhTm2bX5ZlSKTyJBgH2kkpfdik2R2R/Z64rpi2qslUpMo+qkhb+bTUVy0gDA+UaM7CwCEYJv5aieE1tq+vf3ZOY0tMwNJWt/D2QNiceYypLCbF8OcC81EVCXP5QTn1ag6uDcx16Nkx/GwaULC8fMWma1FQA7bmrp7cRiamaY7F6jt9R4Bw0Sq4F5yVOHP9EbAIA]

In [2]:
fs = set('/')
fs.add(f"/a/")


In [3]:
fs.add(f"/a 123)")

In [4]:
for f in fs:
    print(f)

/
/a/
/a 123)


In [7]:
import re
import sys

def read_input():
    filesystem = set("/")
    #initialize the data structure as a "set" with one value (/)
    pwd = "" # current working directory (unix command)

    for line in data.split('\n'): #open(sys.argv[1]).readlines():
        if m := re.match("^\$ cd (.+)$", line): # regex search for $ cd XXXX
            dir = m.group(1) # directory name is matching group 1 (XXXX)
            if dir == "/": # "cd /"
                pwd = "/"
            elif dir == "..": # "cd .."
                pwd = "/".join(pwd.split("/")[:-2]) + "/" # plit the pwd at all ? and take all pieces save the last two
            else: # "cd <directory>""            
                pwd = f"{pwd}{dir}/" # go down in the tree to the next directory: pwd + subdir
                filesystem.add(f"{pwd}") # add new subdir <full-path> to the set
        elif m := re.match("^(\d+) (.+)$", line): # if it's not a command, it's a file
            size, name = m.groups() # the regex has two capture groups: number/size and name
            filesystem.add(f"{pwd}{name} {size}") # store "<full-path> <size>"

    return filesystem # what is returned is a set that has paths to all dirs and files

def is_dir(f):
    return not re.match("\d+$", f)

def is_file_in_dir(f, dir):
    return re.match(f"^{dir}.* \d+$", f)

def file_size(f):
    return int(f.split(" ")[1])

def dir_size(dir, fs):
    return sum(file_size(f) for f in fs if is_file_in_dir(f, dir))
    # for any given f, file size selects the number component and returns it to be summed
    # is file in dir checks to see if the set element is a file by checking to see if the string (f) has a number at hte endfs = read_input()


fs = read_input()
root_size = dir_size("/", fs) # go through the set and add up all the lines with numbers at their end
dir_sizes = [dir_size(d, fs) for d in fs if is_dir(d)] # look at all directories (recursively) and add the size of the files in it

print("1:", sum(s for s in dir_sizes if s <= 100000))
print("2:", min(s for s in dir_sizes if s >= (root_size - 40000000)))

1: 1427048
2: 2940614


In [8]:
fs

{'/',
 '/ctd/',
 '/ctd/bblsqnwl/',
 '/ctd/bblsqnwl/rng/',
 '/ctd/bblsqnwl/rng/jncpmzcs/',
 '/ctd/bblsqnwl/rng/jncpmzcs/tstwmrbr.bmr 36126',
 '/ctd/bblsqnwl/rng/pttvmghm/',
 '/ctd/bblsqnwl/rng/pttvmghm/ctttjn.nzr 38802',
 '/ctd/bblsqnwl/rng/qcjbfggw.tjj 293353',
 '/ctd/bblsqnwl/rng/sgm/',
 '/ctd/bblsqnwl/rng/sgm/djcbdbgr/',
 '/ctd/bblsqnwl/rng/sgm/djcbdbgr/nbb.ztq 254050',
 '/ctd/bblsqnwl/rng/sgm/gqrr/',
 '/ctd/bblsqnwl/rng/sgm/gqrr/jgsqvf.tql 168685',
 '/ctd/bblsqnwl/rng/sgm/gqrr/shmvmqv 302004',
 '/ctd/gzcjrs/',
 '/ctd/gzcjrs/bwnjnmmh.wwd 319972',
 '/ctd/gzcjrs/fzthlhcp.nst 25495',
 '/ctd/gzcjrs/ghjc.nvf 291270',
 '/ctd/gzcjrs/jrlfb.fvz 244664',
 '/ctd/pgqmwn/',
 '/ctd/pgqmwn/zpvthlgp/',
 '/ctd/pgqmwn/zpvthlgp/dqvcqs.zpb 128092',
 '/ctd/pgqmwn/zpvthlgp/nvqsqtjw 140511',
 '/ctd/qzgjp/',
 '/ctd/qzgjp/jgsqvf.tql 23062',
 '/ctd/qzgjp/nbb.ztq 254685',
 '/ctd/qzgjp/pgqmwn/',
 '/ctd/qzgjp/pgqmwn/gffvwmr.pbd 47287',
 '/ctd/qzgjp/shmvmqv 320683',
 '/ctd/qzgjp/shmvmqv.hmg 105368',
 '/ctd/qzgjp/

In [5]:
data = """$ cd /
$ ls
dir ctd
80649 mwcj.pmh
212527 nbb.ztq
dir pgqmwn
152170 scr.smr
17637 snqcgbs.nhv
dir tmw
dir vtm
$ cd ctd
$ ls
dir bblsqnwl
dir gzcjrs
dir pgqmwn
dir qzgjp
dir shmvmqv
$ cd bblsqnwl
$ ls
dir rng
$ cd rng
$ ls
dir jncpmzcs
dir pttvmghm
293353 qcjbfggw.tjj
dir sgm
$ cd jncpmzcs
$ ls
36126 tstwmrbr.bmr
$ cd ..
$ cd pttvmghm
$ ls
38802 ctttjn.nzr
$ cd ..
$ cd sgm
$ ls
dir djcbdbgr
dir gqrr
$ cd djcbdbgr
$ ls
254050 nbb.ztq
$ cd ..
$ cd gqrr
$ ls
168685 jgsqvf.tql
302004 shmvmqv
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd gzcjrs
$ ls
319972 bwnjnmmh.wwd
25495 fzthlhcp.nst
291270 ghjc.nvf
244664 jrlfb.fvz
$ cd ..
$ cd pgqmwn
$ ls
dir zpvthlgp
$ cd zpvthlgp
$ ls
128092 dqvcqs.zpb
140511 nvqsqtjw
$ cd ..
$ cd ..
$ cd qzgjp
$ ls
23062 jgsqvf.tql
254685 nbb.ztq
dir pgqmwn
320683 shmvmqv
105368 shmvmqv.hmg
60805 snqcgbs
$ cd pgqmwn
$ ls
47287 gffvwmr.pbd
$ cd ..
$ cd ..
$ cd shmvmqv
$ ls
80751 crtwwqb
dir dbfm
dir dhhghph
dir pgqmwn
195177 qnm
dir qpbn
dir rwmsqdw
dir shmvmqv
16133 shmvmqv.vwq
187938 tmw
$ cd dbfm
$ ls
dir gbpnjrt
265813 glzz.mjv
108147 nbb.ztq
229253 nvqsqtjw
85743 pgqmwn
261375 qnm
$ cd gbpnjrt
$ ls
179812 mtp.lnf
234550 shmvmqv.qvq
$ cd ..
$ cd ..
$ cd dhhghph
$ ls
dir ghmpb
70514 zqv.ssn
$ cd ghmpb
$ ls
132693 shmvmqv.csb
$ cd ..
$ cd ..
$ cd pgqmwn
$ ls
dir hcbtbr
200292 pgqmwn
67530 sgm
15442 snqcgbs
$ cd hcbtbr
$ ls
249252 dqvcqs.zpb
$ cd ..
$ cd ..
$ cd qpbn
$ ls
dir pmbw
261142 qnm
dir rsfh
239788 sgm
dir wfwl
$ cd pmbw
$ ls
dir gwwdtzp
dir jszzc
32725 rnggjs.cbv
dir sldv
dir tptrlz
$ cd gwwdtzp
$ ls
dir dlvbdn
63066 dslb.mrp
dir shmvmqv
dir tzssf
$ cd dlvbdn
$ ls
190731 nqv.csc
$ cd ..
$ cd shmvmqv
$ ls
223667 nbb.ztq
$ cd ..
$ cd tzssf
$ ls
27296 rlgswwzq.dgj
dir tfdq
206881 wrdqg.pfj
$ cd tfdq
$ ls
26512 wsgh
$ cd ..
$ cd ..
$ cd ..
$ cd jszzc
$ ls
218701 qpz.hdm
$ cd ..
$ cd sldv
$ ls
dir hsbzqtml
3895 jgsqvf.tql
181815 nbb.ztq
294048 shmvmqv.fcm
dir snqcgbs
dir tmw
203484 zfpjq.pvn
dir zlpwdp
$ cd hsbzqtml
$ ls
dir shmvmqv
$ cd shmvmqv
$ ls
103536 nbb.ztq
32635 wmc
$ cd ..
$ cd ..
$ cd snqcgbs
$ ls
95112 dch
$ cd ..
$ cd tmw
$ ls
dir dplm
275451 hsr.zhn
9766 qnm
177383 ztbmqjqb.hwj
$ cd dplm
$ ls
277671 dqvcqs.zpb
$ cd ..
$ cd ..
$ cd zlpwdp
$ ls
315030 snqcgbs.mlz
$ cd ..
$ cd ..
$ cd tptrlz
$ ls
dir jfflz
154685 qbbpstff.fcs
dir sjjgljg
dir zsssqcsb
$ cd jfflz
$ ls
1330 fvldwzn.dbb
227153 nbb.ztq
$ cd ..
$ cd sjjgljg
$ ls
295745 frjnfsgt.fmq
105131 jtbltpv.vhm
174279 pgqmwn
104317 sgm
$ cd ..
$ cd zsssqcsb
$ ls
246361 bnzsv.dqf
dir ggcvqf
dir htqpwzw
249833 jgsqvf.tql
dir lbtjth
dir lcfwb
306768 mmd.gjw
121112 pgqmwn
dir sgm
dir shmvmqv
dir snqcgbs
55055 tmw.rqm
$ cd ggcvqf
$ ls
46573 fcwznd
$ cd ..
$ cd htqpwzw
$ ls
dir bmzvdzsv
193713 fpp.vwd
39898 mfjcnl.szn
65370 msljctr
10685 ppgzbc.trg
94843 shmvmqv.hqn
dir thfdg
$ cd bmzvdzsv
$ ls
74898 pgqmwn.wwh
$ cd ..
$ cd thfdg
$ ls
dir cpsgcll
$ cd cpsgcll
$ ls
244683 gdfw
17203 tmw
$ cd ..
$ cd ..
$ cd ..
$ cd lbtjth
$ ls
dir snqcgbs
$ cd snqcgbs
$ ls
122394 grhbhpw.fsl
55324 jgsqvf.tql
$ cd ..
$ cd ..
$ cd lcfwb
$ ls
138053 nvqsqtjw
$ cd ..
$ cd sgm
$ ls
314313 dqvcqs.zpb
146274 fftqvs.ltt
225372 jtq.rzq
85697 rctzgq.rsr
dir sgm
262221 tqpng
$ cd sgm
$ ls
dir cmvvd
277366 mhs.tpn
77975 nbb.ztq
160966 qrwbvbd.ghf
55949 snqcgbs.qzc
$ cd cmvvd
$ ls
28054 lfsvfsl
165671 tmw.zwh
$ cd ..
$ cd ..
$ cd ..
$ cd shmvmqv
$ ls
51884 bsbwrjnz.dmt
231236 jfmsg.mts
dir qpb
256063 zhmw
65512 zrnhr
$ cd qpb
$ ls
228289 dqvcqs.zpb
$ cd ..
$ cd ..
$ cd snqcgbs
$ ls
dir mnwgf
$ cd mnwgf
$ ls
178834 qgnwvlfd.cps
255140 sls.mnw
9546 tvhbm
281615 vntrb
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd rsfh
$ ls
310413 snqcgbs
$ cd ..
$ cd wfwl
$ ls
dir fmvg
129025 jgsqvf.tql
$ cd fmvg
$ ls
276100 nvqsqtjw
$ cd ..
$ cd ..
$ cd ..
$ cd rwmsqdw
$ ls
46554 qzdmgl
55785 shmvmqv
127982 tfffrg.sbz
$ cd ..
$ cd shmvmqv
$ ls
dir ntstz
dir vprlz
$ cd ntstz
$ ls
304801 nvqsqtjw
28969 wzfb.fjv
$ cd ..
$ cd vprlz
$ ls
dir cmmtnt
dir shmvmqv
$ cd cmmtnt
$ ls
270129 shmvmqv
$ cd ..
$ cd shmvmqv
$ ls
dir sgm
$ cd sgm
$ ls
287889 hghh.mwt
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd pgqmwn
$ ls
85368 qnm
$ cd ..
$ cd tmw
$ ls
dir fnpjwg
dir gdt
dir ggs
dir llh
69470 nvqsqtjw
dir sgm
$ cd fnpjwg
$ ls
161820 dndq.tfl
dir dwmcs
205155 tdccsw.qlq
165352 vvmnpj
$ cd dwmcs
$ ls
121466 dqvcqs.zpb
274171 qbbqmf.ltf
$ cd ..
$ cd ..
$ cd gdt
$ ls
dir cnvtrw
dir gdcvdsv
dir hcn
dir hsmhn
dir jnhqz
78446 pgqmwn.mwr
dir qwfl
dir snqcgbs
$ cd cnvtrw
$ ls
dir hfvzm
$ cd hfvzm
$ ls
305880 tmw.lvw
$ cd ..
$ cd ..
$ cd gdcvdsv
$ ls
69690 jhscd
$ cd ..
$ cd hcn
$ ls
166775 dqvcqs.zpb
294325 nvqsqtjw
$ cd ..
$ cd hsmhn
$ ls
172897 lrjf.dnw
dir npgr
225227 pbcmjg.rhf
234530 pgqmwn.cqb
dir qlz
dir snqcgbs
52808 tnntq.wjt
$ cd npgr
$ ls
141280 qnm
64422 rwv
$ cd ..
$ cd qlz
$ ls
258070 nbb.ztq
$ cd ..
$ cd snqcgbs
$ ls
3723 frcr.jmn
109486 sgm.nsw
114054 shmvmqv.frr
142024 snqcgbs
$ cd ..
$ cd ..
$ cd jnhqz
$ ls
dir gpztwnqm
dir nwpqbf
dir sgm
dir tmw
$ cd gpztwnqm
$ ls
303103 bjbpj
281422 hgp
284044 jgsqvf.tql
dir qzts
133248 sgm
$ cd qzts
$ ls
2428 dqvcqs.zpb
283206 hzs.smg
dir lmlw
$ cd lmlw
$ ls
dir sjrfmpc
$ cd sjrfmpc
$ ls
164708 nbb.ztq
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd nwpqbf
$ ls
192033 dqvcqs.zpb
298220 jqd.bfh
dir mhtsg
100921 snqcgbs
dir zbtb
$ cd mhtsg
$ ls
140813 cgcn.rjn
$ cd ..
$ cd zbtb
$ ls
201213 gjglrmf.dzv
dir grbl
81636 pgqmwn.zfm
265992 pwhbflzn.sch
dir twjrlzpj
$ cd grbl
$ ls
82894 jgsqvf.tql
$ cd ..
$ cd twjrlzpj
$ ls
304952 cwhtqv.zvh
$ cd ..
$ cd ..
$ cd ..
$ cd sgm
$ ls
56329 ftfvfpz.srv
3875 nbb.ztq
4766 shmvmqv.fdt
247272 vbw.hsv
$ cd ..
$ cd tmw
$ ls
dir ddbvd
$ cd ddbvd
$ ls
83200 tnrwdc.dzm
$ cd ..
$ cd ..
$ cd ..
$ cd qwfl
$ ls
dir bcfnz
23440 nvqsqtjw
dir pwb
206984 shmvmqv.fjb
35453 snqcgbs
297136 snqcgbs.fnr
dir tmw
181689 wcbdwwq
$ cd bcfnz
$ ls
dir fnrclwn
dir fslh
258656 gtbhdpt.pth
238261 psbsmvpf.lmt
dir sgm
$ cd fnrclwn
$ ls
30297 crhjfzlp.gtb
297557 gvvnbmt.spr
189301 nvqsqtjw
45159 sndmlvw.lsb
259295 sqd
$ cd ..
$ cd fslh
$ ls
246662 bpvcj.hnf
$ cd ..
$ cd sgm
$ ls
dir jpr
dir tmw
dir wscvtrd
$ cd jpr
$ ls
49005 dqvcqs.zpb
$ cd ..
$ cd tmw
$ ls
201387 sgm
$ cd ..
$ cd wscvtrd
$ ls
77175 blczlqf.pnv
207393 shmvmqv.qbs
$ cd ..
$ cd ..
$ cd ..
$ cd pwb
$ ls
89451 bqcmzzf
207548 gfwgnft.php
dir gsn
dir jtvpnvz
144792 nbb.ztq
dir nzzqj
214209 pgqmwn.sjv
236845 pzvjqqvz
dir sgm
dir snqcgbs
dir vfvttj
$ cd gsn
$ ls
93490 snqcgbs.fbv
$ cd ..
$ cd jtvpnvz
$ ls
dir fdnvff
208857 gpvwzhd
229559 rnc
225519 snqcgbs
162969 tmw
$ cd fdnvff
$ ls
286944 dztqqtf.nmp
dir rwc
181737 snqcgbs
$ cd rwc
$ ls
115290 nbb.ztq
$ cd ..
$ cd ..
$ cd ..
$ cd nzzqj
$ ls
154210 jgsqvf.tql
dir sgm
182438 snqcgbs.wpg
dir zpvghr
$ cd sgm
$ ls
123163 rjgc.ccm
dir rqn
291622 tflv.pqc
$ cd rqn
$ ls
220655 rtbrwmjn.clj
$ cd ..
$ cd ..
$ cd zpvghr
$ ls
288859 dqvcqs.zpb
218036 hfdcrjp
$ cd ..
$ cd ..
$ cd sgm
$ ls
107183 sgm.nsf
225832 sjwn
dir tgf
$ cd tgf
$ ls
258766 nvqsqtjw
$ cd ..
$ cd ..
$ cd snqcgbs
$ ls
56144 whjpg.ffz
$ cd ..
$ cd vfvttj
$ ls
dir ptmggmsl
$ cd ptmggmsl
$ ls
dir psd
dir wdzptvjc
$ cd psd
$ ls
121637 dqvcqs.zpb
$ cd ..
$ cd wdzptvjc
$ ls
dir shmvmqv
$ cd shmvmqv
$ ls
23754 dqmrj.fhh
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd tmw
$ ls
17390 hqz
dir jnsb
31072 nvqsqtjw
dir tmw
144511 vvwjcqr.rtz
$ cd jnsb
$ ls
265907 snqcgbs.swh
$ cd ..
$ cd tmw
$ ls
dir gfncpvw
$ cd gfncpvw
$ ls
dir ctvwgtlh
$ cd ctvwgtlh
$ ls
29825 ccjvs.qqq
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd snqcgbs
$ ls
230355 ccl.ddg
dir csfdmhmb
dir drpgbvzf
313060 hhcpw.nbs
dir pfpn
dir ptwnpdnc
273250 qfsnmbm
dir shmvmqv
244767 tmw.zjg
$ cd csfdmhmb
$ ls
50690 nbb.ztq
282078 vzrw
$ cd ..
$ cd drpgbvzf
$ ls
dir bhwvqf
803 dqvcqs.zpb
49859 hhw.gpd
180202 nbb.ztq
dir nfh
64808 pgqmwn
276746 qnm
dir shmvmqv
dir tmw
$ cd bhwvqf
$ ls
76667 qcgpdwm.sbs
$ cd ..
$ cd nfh
$ ls
23774 jgsqvf.tql
34652 pnp.lvp
120264 zvp
$ cd ..
$ cd shmvmqv
$ ls
47748 cswm.dsr
dir pgqmwn
$ cd pgqmwn
$ ls
109587 qnm
$ cd ..
$ cd ..
$ cd tmw
$ ls
105578 snqcgbs
$ cd ..
$ cd ..
$ cd pfpn
$ ls
194628 wgc
$ cd ..
$ cd ptwnpdnc
$ ls
142456 rvwtfvr.tsc
$ cd ..
$ cd shmvmqv
$ ls
118642 hgqlrt.btl
$ cd ..
$ cd ..
$ cd ..
$ cd ggs
$ ls
231223 hmtp.djc
$ cd ..
$ cd llh
$ ls
dir hbq
dir svt
$ cd hbq
$ ls
157243 fhpsb
36981 rdbqnbfr.cwz
282257 tmw.wwv
$ cd ..
$ cd svt
$ ls
161395 sgm.hdl
$ cd ..
$ cd ..
$ cd sgm
$ ls
247276 fss
dir pgqmwn
68084 pgqmwn.vfl
28475 sgm.swb
dir shmvmqv
223438 snqcgbs.bng
$ cd pgqmwn
$ ls
134640 dqvcqs.zpb
dir nrdd
25431 nvqsqtjw
149751 qhzqw
dir vfmcjhwz
dir zlfhr
$ cd nrdd
$ ls
dir pgqmwn
$ cd pgqmwn
$ ls
57090 nfbd
$ cd ..
$ cd ..
$ cd vfmcjhwz
$ ls
24256 fdzfwfh.rcv
$ cd ..
$ cd zlfhr
$ ls
20504 fgbpndj.wgm
181769 qnm
$ cd ..
$ cd ..
$ cd shmvmqv
$ ls
208988 nbb.ztq
$ cd ..
$ cd ..
$ cd ..
$ cd vtm
$ ls
dir lqvjm
dir msmtcnsj
dir qmtgdn
dir tmw
$ cd lqvjm
$ ls
239772 hnj.zqf
$ cd ..
$ cd msmtcnsj
$ ls
48711 clfl.fsl
245929 ddh.gdq
dir fvlb
241040 nvqsqtjw
dir shmvmqv
305567 vdzzqpz.jfs
$ cd fvlb
$ ls
25669 ddz.tbc
$ cd ..
$ cd shmvmqv
$ ls
283905 nvqsqtjw
$ cd ..
$ cd ..
$ cd qmtgdn
$ ls
246479 pgqmwn
35386 rtf.cdp
217551 sgm.mrz
$ cd ..
$ cd tmw
$ ls
dir qfggmdvd
dir tlv
dir tmw
$ cd qfggmdvd
$ ls
288091 djwfq.wzc
37383 pwnzdj
214119 shmvmqv.wlg
29747 tmw.ntw
146556 zgbd
$ cd ..
$ cd tlv
$ ls
142924 grj.qhw
210672 qhmn
209554 zbcqcb.mhn
141215 zjmrgw.wlc
$ cd ..
$ cd tmw
$ ls
dir fwsgn
dir tmw
$ cd fwsgn
$ ls
dir qbbhrst
306142 sgm
$ cd qbbhrst
$ ls
dir sgm
dir srbn
$ cd sgm
$ ls
80992 bbvqrcf.chp
$ cd ..
$ cd srbn
$ ls
161382 nvqsqtjw
$ cd ..
$ cd ..
$ cd ..
$ cd tmw
$ ls
219508 dqvcqs.zpb
dir tmw
134434 wbvvgqwn.bmh
$ cd tmw
$ ls
209116 pgqmwn"""